# Implementation of an OLS solver

In this exercise, you will implement your own function for solving OLS regression problems in Python.

The function takes the data samples in matrix-form ($X$, $y$) as inputs and returns the minimizing solution $\beta$ as well as the remaining error $\mathcal{L}(\beta)$.

In [18]:
# Imports
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

## Exercise a)

Implement the function.

In [19]:
def fit_parameters(X, y):
    """Compute optimal parameters by least-squares regression.

    Args:
        X (np.ndarray): The input variables, containing intercept variables if required.
        y (np.ndarray): The target variables.

    Returns:
        np.ndarray: The parameter vector (beta)
        float: The remaining loss
    """
    beta = np.linalg.inv(X.T @ X) @ X.T @ y
    y_pred = X @ beta
    loss = np.sum((y - y_pred)**2) 
    return beta, loss

## Exercise b)

For our provided toy data set (*ols-implementation-data.csv*), find the optimal regression parameters with the help of your implementation. Don't forget to add a variable for the intercept parameter!

In [20]:
# Load dataset
# TODO
import statsmodels.api as sm

data = pd.read_csv("ols-implementation-data.csv")
X = data[["x1", "x2"]].values
y = data["y"].values

# Add intercept variables
X_with_intercept = sm.add_constant(X) # add a constant term to the independent values (intercept)

# Find optimal parameter values
beta, loss = fit_parameters(X_with_intercept, y) # TODO
print(f"Parameters by our model: {beta}")
print(f"Loss by our model: {loss:.2f}\n")

Parameters by our model: [47.81880739 -0.25241394  3.38759361]
Loss by our model: 96199.44



## Exercise c)

Repeat b) with the aid of scikit-learn [``LinearRegression``](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) and verify your solution.

In [21]:
# Using scikit-learn
beta_custom, loss_custom = fit_parameters(X_with_intercept, y)

model = LinearRegression(fit_intercept=False)  # We already added the intercept
model.fit(X_with_intercept, y)

beta_sklearn = model.coef_
y_pred = model.predict(X_with_intercept)
loss_sklearn = np.sum((y - y_pred)**2)

print("Parameters by our model:", beta_custom)
print(f"Loss by our model: {loss_custom:.2f}")
print("\nParameters by scikit-learn:", beta_sklearn)
print(f"Loss by scikit-learn: {loss_sklearn:.2f}")

Parameters by our model: [47.81880739 -0.25241394  3.38759361]
Loss by our model: 96199.44

Parameters by scikit-learn: [47.81880739 -0.25241394  3.38759361]
Loss by scikit-learn: 96199.44


## Exercise d)

How much of the total variance can you explain with your model? Compute the R^2 measure. What happens if you forget about the intercept? How does the R^2 measure compare?

In [22]:
# R^2 measure
def calculate_r_squared(y, y_pred):
    ssr = np.sum((y - y_pred)**2)  # Sum of squared residuals
    sst = np.sum((y - np.mean(y))**2)  # Total sum of squares
    r_squared = 1 - (ssr / sst)
    return r_squared

y_pred = X_with_intercept @ beta
r_squared = calculate_r_squared(y, y_pred)

print(f"R^2 with intercept: {r_squared:.4f}")

R^2 with intercept: 0.5540


In [24]:
# Without intercept
def calculate_r_squared_no_intercept(y, y_pred):
    ssr = np.sum((y - y_pred)**2)
    sst_uncorrected = np.sum(y**2)
    r_squared = 1 - (ssr / sst_uncorrected)
    return r_squared

X_no_intercept = X  # Original X without the added intercept column
beta_no_intercept = np.linalg.inv(X_no_intercept.T @ X_no_intercept) @ X_no_intercept.T @ y
y_pred_no_intercept = X_no_intercept @ beta_no_intercept

r_squared_no_intercept = calculate_r_squared_no_intercept(y, y_pred_no_intercept)

print(f"R^2 without intercept: {r_squared_no_intercept:.4f}")

R^2 without intercept: 0.3500


## Exercise e)

The computed R^2 value is not very good (even with the intercept). What could be the reason?

- Non-linear relationships: The linear regression model assumes linear relationships between the independent and dependent variables. If the true relationship is non-linear, a linear model will not capture it well, resulting in a lower $R^2$.